In [79]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import json
import re
import requests
from selenium import webdriver
import time
import numpy as np

## NHL Salary Data

In [86]:
soups={}
for i in range(2011,2023):
    pg=1
    temp=[]
    while True:
        page=requests.get('https://www.capfriendly.com/browse/active/'+str(i)+'?stats-season=2024&hide=team,clauses,age,handed,expiry-status,caphit,skater-stats,goalie-stats&pg='+str(pg))
        soup=bs(page.text, 'html.parser')
        rows=soup.find_all('tr')
        if len(rows)==1:
            break
        pg+=1
        temp.append(soup)

    soups[i]=temp

In [75]:
page=requests.get('https://www.capfriendly.com/browse/active/2016?stats-season=2024&hide=team,clauses,age,handed,expiry-status,caphit,skater-stats,goalie-stats&pg=1')
soup=bs(page.text, 'html.parser')
display(soup)

<!DOCTYPE html>
<html lang="en"><head><title>2015-16 Active NHL Players - CapFriendly - NHL Salary Caps</title><link href="https://cdn2.capfriendly.com/images/favicons/favicon.ico" rel="icon" type="image/x-icon"/><link href="https://cdn2.capfriendly.com/images/favicons/favicon.ico" rel="shortcut icon" type="image/x-icon"/><link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,700,600" rel="stylesheet" type="text/css"/><link href="https://www.capfriendly.com/browse/active/2016" rel="canonical"><link href="https://www.capfriendly.com/browse/active/2016" hreflang="x-default" rel="alternate"/><link href="https://www.capfriendly.com/browse/active/2016" hreflang="en" rel="alternate"/><link href="https://fr.capfriendly.com/browse/active/2016" hreflang="fr" rel="alternate"/><meta charset="utf-8"/><meta content="strict-origin-when-cross-origin" name="referrer"/><meta content="2015-16 NHL players who are currently active in the league" name="description"/><meta content="armchair-g

In [88]:
years=[]
players=[]
salaries=[]
for i in range(2011,2023):
    for soup in soups[i]:
        rows=soup.find_all('tr')
        for j in range(1,len(rows)):
            players.append(rows[j].find('a').text.strip())
            salary=rows[j].find_all('td')
            salaries.append(salary[2].text.strip())
            years.append(i)
        



In [90]:
nhl_salaries = pd.DataFrame({'Year':years, 'Player':players, 'Salary':salaries})
nhl_salaries.to_csv('nhl_salaries.csv', index=False)
display(nhl_salaries)

,Year,Player,Salary
0,2011,Alex Ovechkin,"$9,000,000"
1,2011,Evgeni Malkin,"$9,000,000"
2,2011,Sidney Crosby,"$9,000,000"
3,2011,Eric Staal,"$7,500,000"
4,2011,Brad Richards,"$7,800,000"
...,...,...,...
16915,2022,Jacob MacDonald,"$750,000"
16916,2022,Zach Whitecloud,"$750,000"
16917,2022,Martin Frk,"$750,000"
16918,2022,Jake Bischoff,"$750,000"


In [49]:
display(len(salaries))
sal2023=salaries

819

## NFL Salary Data

In [55]:
soups={}
driver=webdriver.Chrome("C:\Program Files\chromedriver-win64\chromedriver.exe")
for i in range(2011,2023):
    driver.get('https://www.spotrac.com/rankings/nfl/'+str(i)+'/cash/')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    page_source = driver.page_source

    soups[i]=bs(page_source, 'html.parser')

In [57]:
years=[]
players=[]
salaries=[]
for i in range(2011,2023):
    soup=soups[i]
    p = soup.select('.rank-name.player.noborderright h3 a')
    s = soup.select('.rank-value.separator-right.right.noborderright span.info')
    y= [i for x in range(len(p))]
    years.extend(y)
    for player, salary in zip(p, s):
        players.append(player.text.strip())
        salaries.append(salary.text.strip())


nfl_salaries = pd.DataFrame({'Year':years, 'Player':players, 'Salary':salaries})
nfl_salaries.to_csv('nfl_salaries.csv', index=False)
